In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

x_data = cancer_data.data
y_label = cancer_data.target

x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                   y_label,
                                                   test_size=0.2,
                                                   random_state=2)

In [2]:
# 개별 모델 정의
knn_clf = KNeighborsClassifier()
rf_clf = RandomForestClassifier()
ab_clf = AdaBoostClassifier()
dt_clf = DecisionTreeClassifier()
# 최종 모델 정의
'''
parameter 'C' 의미 : Inverse of regularization strength; must be a positive float. 
Like in support vector machines, smaller values specify stronger regularization.
'''
lr_final = LogisticRegression(C=10)

In [3]:
# 개별 모델들 학습
knn_clf.fit(x_train, y_train)
rf_clf.fit(x_train, y_train)
ab_clf.fit(x_train, y_train)
dt_clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [4]:
knn_pred = knn_clf.predict(x_test)
rf_pred = rf_clf.predict(x_test)
ab_pred = ab_clf.predict(x_test)
dt_pred = dt_clf.predict(x_test)

print(f"KNN 정확도 : {accuracy_score(y_test, knn_pred): .4f}")
print(f"Random Forest 정확도 : {accuracy_score(y_test, rf_pred): .4f}")
print(f"AdaBoost 정확도 : {accuracy_score(y_test, ab_pred): .4f}")
print(f"Decision Tree 정확도 : {accuracy_score(y_test, dt_pred): .4f}")

KNN 정확도 :  0.9123
Random Forest 정확도 :  0.9386
AdaBoost 정확도 :  0.9474
Decision Tree 정확도 :  0.9298


In [8]:
# 결과값들을 다시 학습 데이터로 만들기
pred = np.array([knn_pred, rf_pred, ab_pred, dt_pred])
# 이를 feature 포맷으로 만들기 위해서 transpose시키기
pred = np.transpose(pred)
pred.shape

(114, 4)

In [9]:
# 최종 메타 모델로 학습하고 정확도 평가해보기 
# 여기 예시에서는 위에서도 그렇고 test데이터로 평가한 결과값들을 기반으로 만들었지만
# 실제로서는 검증 데이터로 해야함..!
lr_final.fit(pred,y_test)
final_pred = lr_final.predict(pred)
print(f"최종 메타 모델인 Logistic Regression 정확도:{accuracy_score(y_test, final_pred):.4f}")

최종 메타 모델인 Logistic Regression 정확도:0.9649


# CV가 적용된 Stacking 구현 방법
- sklearn 패키지 따로 존재하지만 Python으로 직접 구현해보기

In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings(action='ignore')
def get_stacking_datasets(model, x_train_n, y_train_n, x_test_n, n_folds):
    # CV하기 위해 K-fold 설정
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=42)
    
    # 최종 메타 모델이 사용할 학습 데이터 반환을 위해서 넘파이 배열을 0으로 만들어서 초기화
    train_fold_pred = np.zeros((x_train_n.shape[0], 1)) # 2차원으로
    test_pred = np.zeros((x_test_n.shape[0], n_folds))
    print(model.__class__.__name__, '모델 시작')
    
    for folder_counter, (train_idx, valid_idx) in enumerate(kf.split(x_train_n)):
        # 개별 모델 내부에서 학습하고 1개의 fold로 예측할 데이터 셋 추출
        print(f" Fold 횟수 : {folder_counter+1}")
        x_tr = x_train_n[train_idx]
        y_tr = y_train_n[train_idx]
        x_te = x_train_n[valid_idx]
        
        # 개별 모델이 학습한 후 1개의 fold데이터셋으로 예측값 반환 후 최종 메타모델이 학습할 데이터셋에 첨가
        model.fit(x_tr, y_tr)
        train_fold_pred[valid_idx, :] = model.predict(x_te).reshape(-1,1)
        # 개별 모델이 원본 데이터셋의 검증 데이터셋을 기반으로 예측 결과값 반환 후 최종 메타모델이 검증할 데이터셋에 첨가
        test_pred[:, folder_counter] = model.predict(x_test_n)
    
    # 개별모델안에서 테스트 데이터셋을 기반으로 예측한 결과값들 mean취해주고 2차원으로 바꾸어주기
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean


In [31]:
knn_train, knn_test = get_stacking_datasets(knn_clf, x_train, y_train, x_test, 5)
rf_train, rf_test = get_stacking_datasets(rf_clf, x_train, y_train, x_test, 5)
ab_train, ab_test = get_stacking_datasets(ab_clf, x_train, y_train, x_test, 5)
dt_train, dt_test = get_stacking_datasets(dt_clf, x_train, y_train, x_test, 5)

KNeighborsClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5
RandomForestClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5
AdaBoostClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5
DecisionTreeClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5


In [37]:
# 개별모델이 생성한 학습/검증 데이터 최종 메타 모델이 학습/검증하도록 결합
stack_final_x_train = np.concatenate((knn_train, rf_train, ab_train, dt_train), axis=1)
stack_final_x_test = np.concatenate((knn_test, rf_test, ab_test, dt_test), axis=1)

In [38]:
# 최종 메타모델로 학습
# 최종 메타모델 학습시 label은 원본 데이터의 label(y값)
lr_final.fit(stack_final_x_train, y_train)
stack_final_pred = lr_final.predict(stack_final_x_test)

# 최종 메타모델 성능 평가(비교할 때 원본 데이터의 검증 데이터 label과 비교)
print(f"최종 메타모델 정확도 : {accuracy_score(y_test, stack_final_pred):.4f}")

최종 메타모델 정확도 : 0.9386
